In [ ]:
### For this purposue we have used an Example ofHarmful Algal Bloom (HAB) raster files from the Ocean color earth data website, https://oceancolor.gsfc.nasa.gov/about/projects/cyan/. 
### HAB data are yielded by the Cyanobacteria Assessment Network (CyAN) Project, which is a collaborative project among EPA, NASA, NOAA, and USGS to detect and quantify cyanobacteria algal blooms in U.S. lakes and estuaries using satellite remote sensing.
### For Demonastration we used 2006 data. 
### We applied the Equation  10^DN×0.011714−4.1870866) on each file 
### To convert the DN values into the Cyanobacteria Index (CI)

In [ ]:
from osgeo import gdal
import numpy as np
import os

def cell_statistics(input_tif, output_folder):
    # Open the input tif file
    input_ds = gdal.Open(input_tif, gdal.GA_ReadOnly)

    if input_ds is None:
        print("Error: Could not open input file:", input_tif)
        return

    # Get the raster band
    band = input_ds.GetRasterBand(1)

    # Read the raster data as a numpy array
    data = band.ReadAsArray().astype(float)  # Convert to float

    # Check for NoData values
    nodata_value = band.GetNoDataValue()
    if nodata_value is not None:
        data[data == nodata_value] = np.nan

    # Apply the equation: 10 ^ (input.tif * 0.011714 - 4.1870866)
    result = 10 ** (data * 0.011714 - 4.1870866)

    # Create output filename
    output_filename = "result_" + os.path.basename(input_tif)
    output_path = os.path.join(output_folder, output_filename)

    # Create output file
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(output_path, input_ds.RasterXSize, input_ds.RasterYSize, 1, gdal.GDT_Float32)

    # Write the result to the output file
    output_ds.GetRasterBand(1).WriteArray(result)

    # Set the NoData value
    output_ds.GetRasterBand(1).SetNoDataValue(np.nan)

    # Set the projection and extent information
    output_ds.SetProjection(input_ds.GetProjection())
    output_ds.SetGeoTransform(input_ds.GetGeoTransform())

    # Close the files
    output_ds = None
    input_ds = None

    print("Output file created:", output_path)

# New input and output folders
input_folder = r"E:HAB\Year_2006"
output_folder = r"E:\HAB\Cyno_Index_Converted\2006"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through each tif file in the folder
for file in os.listdir(input_folder):
    if file.endswith(".tif"):
        input_tif = os.path.join(input_folder, file)
        cell_statistics(input_tif, output_folder)
